In [3]:
!pip3 install scikit-learn
!pip3 install torch
!pip3 install torchinfo


In [32]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [5]:
# only for running locally
#dfs = []
#for filename in os.listdir('features_ring'):
#    if filename[-4:] == '.tsv':
#        dfs.append(pd.read_csv('features_ring/' + filename, sep='\t'))
#df = pd.concat(dfs)
#df

In [6]:
# don't need to execute - only to export df for training in cloud
#df.to_csv('contact_df.csv')

In [7]:
!pip3 install google-colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.3 MB/s eta 0:00:00


In [9]:
# only for running on drive for training
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('drive/MyDrive/StructuralBioinformatics/contactprediction/contact_df.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
df.dropna(inplace=True)

# Define ground truth values
#y = df['Interaction'].astype('category')
#y_oneHot = pd.get_dummies(y)
di = {"HBOND": 0, "IONIC": 1, "PICATION": 2, "PIPISTACK": 3, "SSBOND": 4, "VDW": 5}
df = df.replace({"Interaction": di})
y = df['Interaction']
y

0         5
1         0
2         0
3         3
4         0
         ..
454188    5
454189    0
454190    5
454191    0
454192    0
Name: Interaction, Length: 454193, dtype: int64

In [37]:
# Define training features
X = df[['s_rsa', 's_up', 's_down', 's_phi', 's_psi', 's_a1', 's_a2', 's_a3', 's_a4', 's_a5',
        't_rsa', 't_up', 't_down', 't_phi', 't_psi', 't_a1', 't_a2', 't_a3', 't_a4', 't_a5']]

#X = X.fillna({'s_up': X.s_up.mode()[0], 's_down': X.s_down.mode()[0],
#                's_phi': X.s_phi.mode()[0], 's_psi': X.s_psi.mode()[0], 's_a1': X.s_a1.mode()[0],
#                's_a2': X.s_a2.mode()[0], 's_a3': X.s_a3.mode()[0], 's_a4': X.s_a4.mode()[0],
#                's_a5': X.s_a5.mode()[0], 't_up': X.t_up.mode()[0],
#                't_down': X.t_down.mode()[0], 't_phi': X.t_phi.mode()[0], 't_psi': X.t_psi.mode()[0],
#                't_a1': X.t_a1.mode()[0], 't_a2': X.t_a2.mode()[0], 't_a3': X.t_a3.mode()[0],
#                't_a4': X.t_a4.mode()[0], 't_a5': X.t_a5.mode()[0]})

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled

,s_rsa,s_up,s_down,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,t_rsa,t_up,t_down,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5
0,-0.612926,1.212548,1.272426,-0.080637,1.435561,1.032979,0.438314,-1.675155,-0.535573,-1.967293,-0.926898,0.482516,1.366067,-1.502723,1.362867,-0.045132,0.558611,1.029760,0.741296,0.813673
1,0.235709,0.077481,-0.558473,0.268180,-0.783304,1.337499,-1.430106,0.702567,-0.124882,-0.467409,-0.118231,-0.267989,-0.712127,0.257241,-0.791289,1.476577,0.135358,0.690168,0.232221,1.818554
2,1.797971,-1.341352,-0.009203,0.266785,-0.772920,0.928827,0.998308,0.620114,-0.436212,0.636456,1.398591,-1.318697,0.500153,0.283161,-0.812362,1.003587,0.531949,-1.773429,-0.528128,-2.075994
3,-0.096994,1.070665,-0.741563,-1.206618,1.219451,-1.006413,-0.511333,0.894341,-0.687926,0.311533,-0.926898,0.932819,1.192884,-1.816354,1.560677,-0.989173,-0.458974,0.875965,-0.678240,0.242084
4,-0.096994,1.070665,-0.741563,-1.206618,1.219451,-1.006413,-0.511333,0.894341,-0.687926,0.311533,-0.926898,0.932819,1.192884,-1.816354,1.560677,-0.989173,-0.458974,0.875965,-0.678240,0.242084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454188,1.161494,-2.050769,0.173887,-0.373643,-0.294621,-0.389438,1.875560,0.634473,0.904054,1.341807,0.854911,-1.618899,1.019702,0.564392,-0.796048,-1.001773,-0.900002,-0.746055,1.130717,-0.597855
454189,-0.786511,0.503131,-0.558473,0.021218,1.453084,-1.006413,-0.511333,0.894341,-0.687926,0.311533,1.901152,-2.219303,-0.538944,4.190592,-0.494914,-0.386305,2.031665,0.608016,0.890320,1.290104
454190,-0.878125,0.645015,-0.009203,-0.975003,1.163638,-1.237531,-0.465554,1.005515,0.184240,0.563489,-0.433474,1.082920,-1.404858,-1.406819,1.223517,0.237886,1.118505,1.451982,-1.157946,0.939918
454191,1.682248,-1.341352,-1.107742,-1.269405,-0.158335,1.807671,-0.480459,0.265285,-0.555445,1.082368,2.335182,-2.369404,-1.058493,3.708481,-0.372558,-0.386305,2.031665,0.608016,0.890320,1.290104


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=0)
kf = KFold(n_splits=10, shuffle=True)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

class Contacts(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create an instance of the custom dataset for the training data
train_dataset = Contacts(X_train, y_train)
test_dataset = Contacts(X_test, y_test)


dataloader_train = DataLoader(X_train, batch_size=16000, shuffle=False)
dataloader_test = DataLoader(X_test, batch_size=16000, shuffle=False)




In [41]:
class ContactNet(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_layers_dim=[]):
        super().__init__()
        self.layers = nn.ModuleList()
        if len(hidden_layers_dim) == 0:
            self.layers = self.layers.append(nn.Linear(input_dim, num_classes))
        else:
            for layer_idx in range(len(hidden_layers_dim)):
                if layer_idx == 0:  # first layer, from input to hidden
                    self.layers = self.layers.append(nn.Linear(input_dim, hidden_layers_dim[layer_idx]))
                else:  # hidden layers, depending on the input
                    self.layers = self.layers.append(nn.Linear(hidden_layers_dim[layer_idx-1], hidden_layers_dim[layer_idx]))
            self.layers = self.layers.append(nn.Linear(hidden_layers_dim[-1], num_classes))  # final output layer
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=.1)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        if len(self.layers) == 1:
            return self.layers[0](x)
        else:
            for layer in self.layers[:-1]:
                x = F.relu(layer(x))
        return F.log_softmax(x, dim=1)#self.layers[-1](x)

input_size = X.shape[1]  # The number of input features
num_classes = y.shape[1] # The number of output classes

model = ContactNet(input_size, num_classes, [256,256,256])

# Criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

summary(model)

Layer (type:depth-idx)                   Param #
ContactNet                               --
├─ModuleList: 1-1                        --
│    └─Linear: 2-1                       5,376
│    └─Linear: 2-2                       65,792
│    └─Linear: 2-3                       65,792
│    └─Linear: 2-4                       1,542
Total params: 138,502
Trainable params: 138,502
Non-trainable params: 0

In [44]:
class EarlyStopping:
    def __init__(self, tolerance, min_delta):
        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:
                self.early_stop = True

early_stopping = EarlyStopping(tolerance=5, min_delta=0.1)
EPOCHS = 20


loss_train, loss_val = [], []
acc_train, acc_val = [], []
for epoch in range(EPOCHS):

  model.train()
  total_acc_train, total_count_train, n_train_batches, total_loss_train = 0, 0, 0, 0
  for idx in enumerate(dataloader_train):
    out = model(X_train)
    loss = criterion(out, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    total_acc_train += (logits.argmax(1) == label).sum().item()
    total_count_train += label.size(0)
    n_train_batches += 1

  avg_loss_train = total_loss_train/n_train_batches
  loss_train.append(avg_loss_train.item())
  accuracy_train = total_acc_train/total_count_train
  acc_train.append(accuracy_train)

  total_acc_val, total_count_val, n_val_batches, total_loss_val = 0, 0, 0, 0
  with torch.no_grad():
      model.eval()
      for idx, (label, text) in enumerate(dataloader_test):
          logits = model(text)
          loss = criterion(logits, label)
          total_loss_val += loss
          total_acc_val += (logits.argmax(1) == label).sum().item()
          total_count_val += label.size(0)
          n_val_batches += 1
  avg_loss_val = total_loss_val/n_val_batches
  loss_val.append(avg_loss_val.item())
  accuracy_val = total_acc_val/total_count_val
  acc_val.append(accuracy_val)
  if epoch % 1 == 0:
    print(f"epoch: {epoch+1} -> Accuracy: {100*accuracy_train:.2f}%, Loss: {avg_loss_train:.8f}",end=" ---------------- ")
    print(f"Val_Acc: {100*accuracy_val:.2f}%, Val_Loss: {avg_loss_val:.8f}")


  early_stopping(loss_train[-1], loss_val[-1])
  if early_stopping.early_stop == True:
    print("Max Tolerance Reached.")
    break


TypeError: ignored